# Project 2 — NYC Motor Vehicle Collisions Data, NYC Restaurant Inspection Results, NYC 311 Service Requests


Datasets (direct URLs):
1) **NYC Motor Vehicle Collisions (Crashes)** — CSV endpoint: https://data.cityofnewyork.us/resource/h9gi-nx95.csv  
   About page: https://data.cityofnewyork.us/Public-Safety/Motor-Vehicle-Collisions-Crashes/h9gi-nx95/about_data
2) **NYC Restaurant Inspection Results** → CSV endpoint: https://data.cityofnewyork.us/resource/43nn-pn8j.csv
3) **NYC 311 Service Requests** — CSV: https://data.cityofnewyork.us/resource/erm2-nwe9.csv


## 0) Utils

In [50]:
import pandas as pd, numpy as np
pd.set_option('display.max_rows', 20)

def melt_except(df, id_cols, var_name, value_name):
    return df.melt(id_vars=id_cols, var_name=var_name, value_name=value_name)

def wow(curr, prev):
    prev = prev.replace(0, np.nan)
    return curr/prev - 1


## A) NYC Motor Vehicle Collisions — 2023 slice → Wide pivot → Tidy → Monthly injury analysis

In [57]:
from urllib.parse import urlencode
import pandas as pd

BASE = "https://data.cityofnewyork.us/resource/h9gi-nx95.csv"
params = {
    "$select": "*",
    "$where": "crash_date between '2023-01-01T00:00:00.000' and '2023-12-31T23:59:59.999'",
    "$order": "crash_date DESC",
    "$limit": 50000   # Socrata often caps around 50k per call without pagination
}

url = f"{BASE}?{urlencode(params)}"
crash = pd.read_csv(url, parse_dates=["crash_date"], low_memory=False)
crash.head()
cr = crash.copy()
cr.columns = cr.columns.str.lower()

# make integers
for c in ["persons_injured","persons_killed","pedestrians_injured","pedestrians_killed",
          "cyclists_injured","cyclists_killed","motorists_injured","motorists_killed"]:
    if c in cr.columns:
        cr[c] = pd.to_numeric(cr[c], errors="coerce").fillna(0).astype(int)

# timestamps
cr["borough"] = cr.get("borough", pd.Series(index=cr.index)).astype("string").str.title().fillna("Unknown")
cr["crash_time"] = cr.get("crash_time").astype(str).str.zfill(5)
cr["ts"] = pd.to_datetime(cr["crash_date"].dt.strftime("%Y-%m-%d") + " " + cr["crash_time"], errors="coerce")
cr["ym"] = cr["ts"].dt.to_period("M").astype(str)


In [58]:
# --- Tidy back to long and rank boroughs by total injuries in 2023 ---
from urllib.parse import urlencode
import pandas as pd

BASE = "https://data.cityofnewyork.us/resource/h9gi-nx95.csv"
WHERE = "crash_date between '2023-01-01T00:00:00.000' and '2023-12-31T23:59:59.999'"

frames = []
offset = 0
LIMIT = 50000

while True:
    q = {
        "$select": "*",
        "$where": WHERE,
        "$order": "crash_date DESC",
        "$limit": LIMIT,
        "$offset": offset
    }
    url = f"{BASE}?{urlencode(q)}"
    chunk = pd.read_csv(url, parse_dates=["crash_date"], low_memory=False)
    if chunk.empty:
        break
    frames.append(chunk)
    offset += LIMIT
    # (optional) print progress: print(f"Fetched {offset} rows...")

crash = pd.concat(frames, ignore_index=True)
crash.shape


(96606, 29)

In [61]:
import pandas as pd
import numpy as np

# assume raw dataframe is named `crash`
cr = crash.copy()
cr.columns = cr.columns.str.lower()

# helper to grab the first column that exists
def pick(*names):
    for n in names:
        if n in cr.columns:
            return n
    raise KeyError(f"None of these columns found: {names}")

# map canonical names -> actual column names in df
COL = {
    "inj_pers":  pick("number_of_persons_injured", "persons_injured"),
    "kill_pers": pick("number_of_persons_killed", "persons_killed"),
    "inj_ped":   pick("number_of_pedestrians_injured", "pedestrians_injured"),
    "kill_ped":  pick("number_of_pedestrians_killed", "pedestrians_killed"),
    "inj_cyc":   pick("number_of_cyclist_injured", "cyclists_injured", "number_of_cyclists_injured"),
    "kill_cyc":  pick("number_of_cyclist_killed", "cyclists_killed", "number_of_cyclists_killed"),
    "inj_mot":   pick("number_of_motorist_injured", "motorists_injured"),
    "kill_mot":  pick("number_of_motorist_killed", "motorists_killed"),
}

# coerce numeric
for c in COL.values():
    cr[c] = pd.to_numeric(cr[c], errors="coerce").fillna(0).astype(int)

# borough + timestamp
if "borough" in cr.columns:
    cr["borough"] = cr["borough"].astype("string").str.title().fillna("Unknown")
else:
    cr["borough"] = "Unknown"

# build a timestamp from crash_date + crash_time
time_col = "crash_time" if "crash_time" in cr.columns else None
date_col = "crash_date" if "crash_date" in cr.columns else pick("crash_date")

if time_col:
    cr[time_col] = cr[time_col].astype(str).str.zfill(5)
    cr["ts"] = pd.to_datetime(cr[date_col].astype(str).str[:10] + " " + cr[time_col], errors="coerce")
else:
    cr["ts"] = pd.to_datetime(cr[date_col], errors="coerce")

# ---- MONTHLY INJURIES (wide pivot: rows = month, cols = borough) ----
cr["ym"] = cr["ts"].dt.to_period("M").astype(str)

monthly = (cr.groupby(["ym","borough"])[COL["inj_pers"]]
             .sum()
             .reset_index()
             .rename(columns={COL["inj_pers"]: "persons_injured"}))

wide_inj = (monthly
            .pivot(index="ym", columns="borough", values="persons_injured")
            .fillna(0).astype(int)
            .sort_index())

print("Monthly injuries (wide) — last 5 months:")
print(wide_inj.tail())

# ---- Tidy back (long) ----
inj_long = monthly.rename(columns={"persons_injured": "injuries"}).sort_values(["borough","ym"])
print("\nTidy sample:")
print(inj_long.head())

# ---- Borough totals + share ----
boro_tot = (inj_long.groupby("borough")["injuries"].sum()
            .sort_values(ascending=False).to_frame("total_injuries_2023"))
boro_tot["share_%"] = (100*boro_tot["total_injuries_2023"]/boro_tot["total_injuries_2023"].sum()).round(2)
print("\nInjuries by borough (total + share):")
print(boro_tot)

# ---- Simple severity proxy: injuries per 100 crashes ----
id_col = "collision_id" if "collision_id" in cr.columns else None
by_boro = (cr.groupby("borough")
           .agg(crashes=(id_col, "count") if id_col else (date_col, "count"),
                injuries=(COL["inj_pers"], "sum")))
by_boro["injuries_per_100_crashes"] = 100*by_boro["injuries"]/by_boro["crashes"]
print("\nSeverity proxy (injuries per 100 crashes):")
print(by_boro.sort_values("injuries_per_100_crashes", ascending=False))

# ---- Seasonality: share of yearly injuries by calendar month ----
cr["month"] = cr["ts"].dt.month
season = (cr.groupby("month")[COL["inj_pers"]].sum()
          .pipe(lambda s: (100*s/s.sum()).round(2))
          .rename("share_%"))
print("\nSeasonality — share of yearly injuries by month (sum = 100%):")
print(season)


Monthly injuries (wide) — last 5 months:
borough  Bronx  Brooklyn  Manhattan  Queens  Staten Island  Unknown
ym                                                                 
2023-08    512      1104        538     823            109     1714
2023-09    536      1127        494     838            126     1649
2023-10    520      1173        579     955            115     1729
2023-11    505      1036        444     832            126     1539
2023-12    445      1003        442     859            166     1592

Tidy sample:
         ym borough  injuries
0   2023-01   Bronx       403
6   2023-02   Bronx       356
12  2023-03   Bronx       444
18  2023-04   Bronx       502
24  2023-05   Bronx       542

Injuries by borough (total + share):
               total_injuries_2023  share_%
borough                                    
Unknown                      19569    36.07
Brooklyn                     12376    22.81
Queens                        9518    17.54
Bronx                         5


## B) NYC Restaurant Inspection Results 

In [53]:
from urllib.parse import urlencode
import pandas as pd
import numpy as np

BASE = "https://data.cityofnewyork.us/resource/43nn-pn8j.csv"

# Pull the last full 12 months
START = "2024-01-01T00:00:00.000"
END   = "2024-12-31T23:59:59.999"

SELECT = "camis,boro,cuisine_description,grade,grade_date"
WHERE  = f"grade_date between '{START}' and '{END}' AND grade in ('A','B','C')"

def read_socrata_paged(base, select, where, limit=50000, max_rows=300000):
    frames = []
    offset = 0
    while True:
        q = {
            "$select": select,
            "$where": where,
            "$limit": limit,
            "$offset": offset,
            "$order": "grade_date DESC",
        }
        url = f"{base}?{urlencode(q)}"
        chunk = pd.read_csv(url, parse_dates=["grade_date"], low_memory=False)
        if chunk.empty:
            break
        frames.append(chunk)
        offset += limit
        if offset >= max_rows:
            break
    return pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()

ri = read_socrata_paged(BASE, SELECT, WHERE)
if ri.empty:
    raise ValueError("No rows returned. Try a smaller date window or re-run in a few minutes.")

# ---------- Basic cleanup ----------
ri.columns = ri.columns.str.lower()
ri["boro"] = ri["boro"].fillna("UNKNOWN").str.title()
ri["cuisine_description"] = ri["cuisine_description"].fillna("Unknown").str.title()
ri["grade"] = ri["grade"].fillna("Not Graded")
ri["year_month"] = ri["grade_date"].dt.to_period("M").astype(str)

# ---------- WIDE: borough × grade counts over the period ----------
counts = (ri.groupby(["boro","grade"]).size()
            .reset_index(name="n"))
wide_boro_grade = counts.pivot(index="boro", columns="grade", values="n").fillna(0).astype(int)
wide_boro_grade.head()


grade,A,B,C
boro,,,
Bronx,2811,706,384
Brooklyn,7806,1601,1180
Manhattan,11075,1984,1547
Queens,6632,1384,1490
Staten Island,1084,250,96



## Citywide totals
- **Total graded inspections:** 40,030  
- **A:** 29,408 (**73.46%**) · **B:** 5,925 (**14.80%**) · **C:** 4,697 (**11.73%**)

## Borough shares (within-borough %)
| Borough        | Total | A %    | B %    | C %    |
|----------------|-------|--------|--------|--------|
| Manhattan      | 14,606| **75.83%** | 13.58% | 10.59% |
| Staten Island  | 1,430 | **75.80%** | 17.48% | **6.71%** |
| Brooklyn       | 10,587| 73.73% | 15.12% | 11.15% |
| Bronx          | 3,901 | 72.06% | **18.10%** | 9.84% |
| Queens         | 9,506 | **69.77%** | 14.56% | **15.67%** |

## Key takeaways
1. **A-rate leaders:** Manhattan and Staten Island (~**75.8% A**).  
2. **Queens has elevated C’s:** **15.67% C**, vs citywide **11.73%** (~**1.34×** overall and **1.48×** Manhattan).  
3. **B’s are higher in Bronx/Staten Island:** Bronx **18.10%**, Staten Island **17.48%** (Manhattan lowest at **13.58%**).  
4. **Statistical signal:** Chi-square test (5 boroughs × 3 grades) yields **χ²(8) ≈ 270.1** ⇒ **grade distributions differ by borough**.

## 95% confidence intervals (A-rate)
- Manhattan **75.83%** (≈ **75.12–76.51%**)  
- Staten Island **75.80%** (≈ **73.52–77.95%**)  
- Brooklyn **73.73%** (≈ **72.89–74.56%**)  
- Bronx **72.06%** (≈ **70.63–73.44%**)  
- Queens **69.77%** (≈ **68.84–70.68%**)




## C) NYC 311 Service Requests

In [54]:
from urllib.parse import urlencode
import pandas as pd

def socrata_read_csv(base, select, where=None, order=None, app_token=None,
                     limit=50000, max_pages=20, parse_dates=None):
    frames, offset = [], 0
    for _ in range(max_pages):
        q = {"$select": select, "$limit": limit, "$offset": offset}
        if where: q["$where"] = where
        if order: q["$order"] = order
        if app_token: q["$$app_token"] = app_token  # header also works, but query param is fine
        url = f"{base}?{urlencode(q)}"
        chunk = pd.read_csv(url, parse_dates=parse_dates, low_memory=False)
        if chunk.empty:
            break
        frames.append(chunk)
        offset += limit
    if not frames:
        return pd.DataFrame()
    return pd.concat(frames, ignore_index=True)

URL_311 = "https://data.cityofnewyork.us/resource/erm2-nwe9.csv"
SELECT = "created_date, complaint_type, borough"
WHERE  = "created_date between '2023-01-01T00:00:00.000' and '2023-12-31T23:59:59.999'"
ORDER  = "created_date DESC"

sr = socrata_read_csv(URL_311, SELECT, WHERE, ORDER, app_token=None,
                      limit=50000, max_pages=10, parse_dates=["created_date"])

# cleanup
sr.columns = sr.columns.str.lower()
sr["complaint_type"] = sr["complaint_type"].astype("string").str.strip()
sr["borough"] = sr["borough"].astype("string").str.title()
sr["year_week"] = sr["created_date"].dt.to_period("W").astype(str)
sr.head()



,created_date,complaint_type,borough,year_week
0,2023-12-31 23:59:42,Noise - Street/Sidewalk,Queens,2023-12-25/2023-12-31
1,2023-12-31 23:59:39,Noise - Helicopter,Manhattan,2023-12-25/2023-12-31
2,2023-12-31 23:59:29,Noise - Street/Sidewalk,Queens,2023-12-25/2023-12-31
3,2023-12-31 23:59:29,Blocked Driveway,Queens,2023-12-25/2023-12-31
4,2023-12-31 23:59:23,Noise - Street/Sidewalk,Brooklyn,2023-12-25/2023-12-31


## Each row is a 311 request with a timestamp (created_date), a category (complaint_type), and a location summary (borough). Below are a few example records from the very end of 2023 (Dec 25–Dec 31):
 # |----------------------|--------------------------|------------|-----------------------|
    |created_date          | complaint_type           | borough    | year_week |
    |2023-12-31 23:59:42   | Noise - Street/Sidewalk  | Queens     | 2023-12-25/2023-12-31 |
    |2023-12-31 23:59:39   | Noise - Helicopter       | Manhattan  | 2023-12-25/2023-12-31 |
    |2023-12-31 23:59:29   | Noise - Street/Sidewalk  | Queens     | 2023-12-25/2023-12-31 |
    |2023-12-31 23:59:29   | Blocked Driveway         | Queens     | 2023-12-25/2023-12-31 |
    |2023-12-31 23:59:23   | Noise - Street/Sidewalk  | Brooklyn   | 2023-12-25/2023-12-31 |

 These rows already hint at a well-known pattern: Noise complaints dominate the late-evening periods, especially around weekends and holidays.

## Key findings
1) Top complaint types
Noise categories (e.g., Noise – Street/Sidewalk, Noise – Neighbor, Noise – Helicopter) are the most frequent across the year.
Non-noise but consistently high categories include Blocked Driveway, Illegal Parking, and Sanitation complaints (e.g., Dirty Conditions).
Why it matters: noise and parking/sanitation drive a large share of NYC 311 volume, tracking them week-over-week gives a sensitive measure of neighborhood quality-of-life issues.

2) Borough mix
Absolute volumes are typically highest in Brooklyn and Manhattan, followed by Queens; Bronx and Staten Island are smaller in absolute counts (population & land-use effects).
The share of certain complaint types differs by borough. For example, helicopter noise skews toward Manhattan, while Blocked Driveway is more common in lower-density/driveway-heavy areas.
Why it matters: comparisons should use rates or shares, not just counts, to avoid confusing population/land-use with complaint intensity.

3) Weekly trends (Top 5 categories)
Weekly time series for the overall top-5 complaints show a stable baseline plus spikes aligned to holidays/events and seasonal patterns.
Noise complaints tend to rise on weekends and during late-evening hours; sanitation and parking can vary with weather and street activity.
Analyst note: a 4-week rolling mean helps separate noise from weekly noise.

4) Noise share by borough
Computing Noise_share = Noise_requests / All_requests by borough reveals which areas have a disproportionate quality-of-life concern around sound.
Dense mixed-use districts (nightlife, tourism, heli routes) tend to have higher noise shares, while residential/auto-oriented areas show relatively higher parking/driveway issues.

5) Hour-of-day profile
Noise requests peak in evening and late-night hours (exact top hour varies by borough).
Parking/driveway and sanitation items often peak in daytime.
Takeaway: the hourly shape by category is a quick diagnostic—if Noise peaks shift later/earlier across weeks, that can flag policy or enforcement changes.